In [1]:
from glob import glob
from tqdm import tqdm
import numpy as np
import cv2
import re
import function as f
import os

In [2]:
# 먼저 padding시켜서 정사각형으로 만들기
def pad_to_square(a, LR, pad_value=0): # image, 좌우, 채워넣을 값
    m = a.reshape((a.shape[0], -1))
    padded = pad_value * np.ones(2 * [max(m.shape)], dtype=m.dtype)
    if  LR != 0 : # 좌측이면
        padded[0:m.shape[0], 0:m.shape[1]] = m
    else : # 우측이면
        padded[:,-m.shape[1]:] = m
    return padded

In [3]:
def get_resize(image, dim, LR): # image, 원하는 타겟 dimension, 좌우
    image_square = pad_to_square(image, LR)
    image_resize = cv2.resize(image_square, dim, interpolation = cv2.INTER_AREA)
    return image_resize

In [4]:
def one_hot_encoding(array): # 0,1,2,3값을 각각 one-hot-encoding하여 4개의 레이어로 만들기
    array = array.astype(int)
    return np.stack(((array==0)*1,(array==1)*1,(array==2)*1,(array==3)*1))

def one_hot_reverse(segs): # 다시 0,1,2,3으로 값 만들때
    segment = segs[1].copy()
    segment[segs[0]==1] = 0    
    segment[segs[2]==1] = 2    
    segment[segs[3]==1] = 3
    return segment

In [5]:
# density label 있는 것들
list_segs = glob("/data1/home/jyhwang/numpy/segment/*/*")
#list_images = [re.sub("segment","image",i) for i in list_segs]
len(list_segs)

14069

In [6]:
# 완전 전체
list_total = glob("/data1/home/jyhwang/numpy/image/*/*")
len(list_total)

43533

In [7]:
import random
random.shuffle(list_total)

In [271]:
# density label 있는 것들
dim = (256, 256)
for i in tqdm(range(len(list_segs)),mininterval=5,miniters=50):
    image = list_images[i]
    segment = list_segs[i]
    LR = np.load(image)[0,0]
    final = np.load(image)
    final = final/final.max()
    np.save(re.sub("image","image_"+str(dim[0])+'x'+str(dim[1]),image), get_resize(final, dim = dim, LR = LR))
    np.save(re.sub("segment","segment_"+str(dim[0])+'x'+str(dim[1]),segment), one_hot_encoding(get_resize(np.load(segment), dim = dim, LR = LR)))

100%|██████████| 14069/14069 [17:23<00:00, 13.48it/s]


In [11]:
# 위에서 돌린 것 말고 나머지 전체
dim = (256, 256)
for i in tqdm(range(len(list_total)),mininterval=5,miniters=50):
    image = list_total[i]
    target = re.sub("image","image_"+str(dim[0])+'x'+str(dim[1]),image)
    if os.path.exists(target) is False:
        final = np.load(image)
        final = final/final.max()
        try:
            np.save(target, get_resize(final, dim = dim, LR = final[0,0]))
        except:
            pass

100%|██████████| 43533/43533 [08:16<00:00, 87.72it/s]  


In [ ]:
dim = (512, 512)
for i in tqdm(range(len(list_segs)),mininterval=5,miniters=50):
    image = list_images[i]
    segment = list_segs[i]
    LR = np.load(image)[0,0]
    final = np.load(image)
    final = final/final.max()
    np.save(re.sub("image","image_"+str(dim[0])+'x'+str(dim[1]),image), get_resize(final, dim = dim, LR = LR))
    #np.save(re.sub("segment","segment_"+str(dim[0])+'x'+str(dim[1]),segment), one_hot_encoding(get_resize(np.load(segment), dim = dim, LR = LR)))